# import

In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
import gensim
assert gensim.models.word2vec.FAST_VERSION > -1
from gensim.models import Word2Vec
import datetime
import requests
import re
from bs4 import BeautifulSoup

# Data 준비하기

In [28]:
df_activity = pd.read_csv('activity_process.csv') # 대외활동 데이터
df_activity = df_activity.dropna()
def remove_special_characters(text):
    pattern = r'[^A-Za-z0-9가-힣\s]'  # 특수문자 패턴 정의
    return re.sub(pattern, '', text)

df_activity['activity_name'] = df_activity['activity_name'].apply(remove_special_characters)
id_to_name = pd.Series(df_activity.activity_name.values, index = df_activity.id.values)
name_to_id = pd.Series(df_activity.id.values, index = df_activity.activity_name).to_dict()


df_activity.head(5)

,Unnamed: 0,id,activity_name,recruit,prefer,activity_benefit,interests,fields,경영/컨설팅/마케팅,경제/금융,...,우대역량_기타,취재/인터뷰/기사 경험,컴퓨터 활용능력,콘텐츠 기획/제작 경험,타 대외활동 경험,파워블로거/SNS,행사 기획 경험,휴학 중/시간 투자 가능,모집 인원,모집인원규모
0,0,120014,인천 대외활동청년 대학생을 위한 Mindset 명상캠프,15명,우대역량_기타,행사 참여,"교육, 창업/자기계발","fields_기타, 강연",0,0,...,1,0,0,0,0,0,0,0,15,소
1,1,120021,만월종합사회복지관 도담멘토링 멘토 모집,00명,멘토링/수업 진행 경험,"수료증 및 인증서, 봉사활동시간","교육, 사회공헌/교류","봉사단-국내, 멘토링",0,0,...,0,0,0,0,0,0,0,0,99,중
2,2,120025,대외활동 한국전통미술융합진흥원 문화디지털 도슨트를 모집합니다,00명,"파워블로거/SNS, 멘토링/수업 진행 경험, 콘텐츠 기획/제작 경험","활동비, 수료증 및 인증서","문화/역사, 콘텐츠","서포터즈, fields_기타",0,0,...,0,0,0,1,0,1,0,0,99,중
3,3,120027,원주 KB대학생경제금융교육봉사단 폴라리스 20기 모집,10명,"사진/영상/디자인 툴, 멘토링/수업 진행 경험, 리더십, 휴학 중/시간 투자 가능","활동비, 수료증 및 인증서","교육, 사회공헌/교류","봉사단-국내, 멘토링",0,0,...,0,0,0,0,0,0,0,1,10,소
4,4,120033,경기도 공익활동 시민기록자 양성교육 공기공익활동을 기록하다 입문과정 참여자 모집21...,30명,"파워블로거/SNS, 사진/영상/디자인 툴, 취재/인터뷰/기사 경험, 멘토링/수업 진...","행사 참여, 수료증 및 인증서","언론/미디어, 사회공헌/교류","강연, 멘토링",0,0,...,0,1,0,0,0,1,0,1,30,중


# 필요없는 열제거

In [29]:
df_activity=df_activity.drop(["recruit"],axis='columns')
df_activity=df_activity.drop(["Unnamed: 0"],axis='columns')
df_activity=df_activity.drop(["activity_benefit"],axis='columns')
df_activity=df_activity.drop(["prefer"],axis='columns')
df_activity=df_activity.drop(["fields"],axis='columns')
df_activity=df_activity.drop(["interests"],axis='columns')
df_activity=df_activity.drop(["모집 인원"],axis='columns')
df_activity.head(3)

,id,activity_name,경영/컨설팅/마케팅,경제/금융,과학/공학/기술/IT,교육,기타,디자인/사진/예술/영상,문화/역사,뷰티/미용/화장품,...,외국어,우대역량_기타,취재/인터뷰/기사 경험,컴퓨터 활용능력,콘텐츠 기획/제작 경험,타 대외활동 경험,파워블로거/SNS,행사 기획 경험,휴학 중/시간 투자 가능,모집인원규모
0,120014,인천 대외활동청년 대학생을 위한 Mindset 명상캠프,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,소
1,120021,만월종합사회복지관 도담멘토링 멘토 모집,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,중
2,120025,대외활동 한국전통미술융합진흥원 문화디지털 도슨트를 모집합니다,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,1,0,0,중


# train set, test set 분리

In [30]:
#df_activity_train, df_activity_test = train_test_split(df_activity, random_state = 15688, test_size=0.30)
df_activity_train = df_activity


In [100]:
df_activity_train.head(3)

,id,activity_name,경영/컨설팅/마케팅,경제/금융,과학/공학/기술/IT,교육,기타,디자인/사진/예술/영상,문화/역사,뷰티/미용/화장품,...,외국어,우대역량_기타,취재/인터뷰/기사 경험,컴퓨터 활용능력,콘텐츠 기획/제작 경험,타 대외활동 경험,파워블로거/SNS,행사 기획 경험,휴학 중/시간 투자 가능,모집인원규모
0,120014,인천 대외활동청년 대학생을 위한 Mindset 명상캠프,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,소
1,120021,만월종합사회복지관 도담멘토링 멘토 모집,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,중
2,120025,대외활동 한국전통미술융합진흥원 문화디지털 도슨트를 모집합니다,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,1,0,0,중


# 같은 대외활동끼리 묶기

In [46]:
# 비교할 때 제외할 열들의 이름 리스트
def data_splitter(df):
    exclude_cols = ['id', 'activity_name']

    grouped_cols = [col for col in df_activity.columns if col not in exclude_cols]  # 제외할 열을 제외한 열들의 리스트

    df_grouped = df_activity.groupby(grouped_cols).apply(lambda x: x.id.tolist())
    
    # 결과값을 str 타입으로 저장한 이중 배열
    array_of_groups = df_grouped.apply(lambda x: [str(val) for val in x]).tolist()
    return array_of_groups

In [47]:
pd.options.mode.chained_assignment = None
activity_ls = data_splitter(df_activity_train)

# 데이터 랜덤으로 섞고, 유효한 데이터 골라내기

In [48]:
for activity in activity_ls:
    random.shuffle(activity)

In [49]:
filtered_array = [group for group in activity_ls if len(group) > 1]

In [50]:
len(filtered_array)

672

In [104]:
print(filtered_array)

[['93650', '90649'], ['94412', '96116'], ['88183', '134497'], ['124119', '85051', '97688', '93148'], ['93483', '80507', '79783'], ['125394', '122045'], ['127333', '95325', '87241'], ['83236', '98798'], ['92203', '91596', '89811'], ['94941', '93155'], ['133993', '92395'], ['128082', '124299', '121379', '130805'], ['128919', '98997'], ['98750', '95882'], ['133223', '80280', '128621'], ['135119', '125180'], ['89550', '89553', '89544'], ['86049', '86050'], ['121599', '96873'], ['126718', '80196'], ['94453', '94456'], ['127001', '128721'], ['124817', '94222'], ['121627', '125985'], ['91880', '95519'], ['85736', '97848'], ['97218', '82313'], ['133983', '131551', '130608'], ['94748', '94750'], ['81611', '83022', '91969'], ['79658', '92889'], ['125780', '87235', '130240', '121447', '83029'], ['126741', '130003'], ['98438', '85983', '128880', '125573', '127919', '125176', '127530', '131978', '127529', '123468'], ['76760', '122236'], ['99364', '120373', '86497', '133267', '92380', '79289', '1298

# Model 생성 및 학습

In [82]:
model = Word2Vec(
    sentences = filtered_array, # 전처리된 리스트를 파라미터로 사용
    epochs = 100, # epoch
    min_count = 1, # 
    vector_size = 100, # hidden layer 사이즈
    sg = 1, # skip-gram을 사용하면 1
    hs = 0, # negative sampling을 사용하면 0
    negative = 5, # 0 이상일 경우 negative sampling을 사용함.
    window=5,  # 주변 단어 창 크기
    workers=4  # 사용할 CPU 스레드 수
)

model.save('item2vec_activity')

In [83]:
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
model = Word2Vec.load('item2vec_activity')
word_vectors = model.wv

# 추천

In [84]:
def recommender(positive_list=None, negative_list=None, topn=20):
    recommend_activity_ls = []
    
    for activityId, prob in model.wv.most_similar_cosmul(positive=positive_list, negative=negative_list, topn=topn):
        recommend_activity_ls.append(activityId)
    return recommend_activity_ls


In [106]:
reco = [99786]
reco_str = [str(num) for num in reco]
recommandations = recommender(positive_list=reco_str, topn=5)
reco_int = [int(value) for value in recommandations] # str -> int
print(reco_int)

[97088, 92053, 127779, 84633, 78482]


In [107]:
# 입력 데이터
filtered_activity_names = df_activity.loc[df_activity['id'].isin(reco), 'activity_name'].tolist()
print(filtered_activity_names)

['송파런 대학생 멘토링 멘토단 모집']


In [108]:
# 출력 데이터
filtered_activity_names = df_activity.loc[df_activity['id'].isin(reco_int), 'activity_name'].tolist()
print(filtered_activity_names)

['23년도 여주시 대학생 멘토링 멘토단 모집', '2022 EBS 멘토링 온라인 멘토 모집', '2022년 EBS 온라인 멘토 모집', 'EBS지원센터 2022 EBS 온라인 멘토 모집', '셰어익스피어 대학생 멘토 모집']


In [109]:
print(model.wv.most_similar(["99786"]))

[('97088', 0.9933083653450012), ('92053', 0.9929162859916687), ('127779', 0.9925497770309448), ('84633', 0.9923624396324158), ('78482', 0.9913104176521301), ('77822', 0.9902252554893494), ('128925', 0.9876944422721863), ('81626', 0.9825983047485352), ('82300', 0.97996586561203), ('123328', 0.9758858680725098)]
